In [1]:
%run ../utils.ipynb

[2024-01-08 19:17:13,603 INFO] [numexpr.utils.utils._init_num_threads] Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.


In [2]:
import os
import sys

from dotenv import load_dotenv, find_dotenv

from yfpy import Data
from yfpy.logger import get_logger
from yfpy.query import YahooFantasySportsQuery

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import json

[2024-01-08 19:17:13,851 DEBUG] [matplotlib.__init__.wrapper] matplotlib data path: C:\Users\mikem\anaconda3\Lib\site-packages\matplotlib\mpl-data
[2024-01-08 19:17:13,855 DEBUG] [matplotlib.__init__.wrapper] CONFIGDIR=C:\Users\mikem\.matplotlib
[2024-01-08 19:17:13,858 DEBUG] [matplotlib.__init__.<module>] interactive is False
[2024-01-08 19:17:13,858 DEBUG] [matplotlib.__init__.<module>] platform is win32
[2024-01-08 19:17:13,940 DEBUG] [matplotlib.__init__.wrapper] CACHEDIR=C:\Users\mikem\.matplotlib
[2024-01-08 19:17:13,944 DEBUG] [matplotlib.font_manager.font_manager._load_fontmanager] Using fontManager instance from C:\Users\mikem\.matplotlib\fontlist-v330.json
[2024-01-08 19:17:13,944 DEBUG] [matplotlib.font_manager.font_manager._load_fontmanager] Using fontManager instance from C:\Users\mikem\.matplotlib\fontlist-v330.json


### Environment variables

In [3]:

project_dir = os.environ.get("BASE_PATH")

In [4]:
# if not sys.path[0] == project_dir:
    # sys.path.insert(0, str(project_dir))

In [5]:
load_dotenv(dotenv_path=f"{project_dir}/auth/.env")

True

### Set directories

In [6]:
auth_dir = f"{project_dir}/auth"

In [7]:
data_dir = f"{project_dir}/data"

In [8]:
yfpy_data_dir = f"{project_dir}/output"

In [9]:
data = Data(yfpy_data_dir)

In [10]:
league_ids = get_league_ids()

In [11]:
game_keys_df = get_game_keys()

In [12]:

print(game_keys_df)


    season  code
16    2015   348
17    2016   359
18    2017   371
19    2018   380
20    2019   390
21    2020   399
22    2021   406
23    2022   414
24    2023   423


In [13]:
league_ids

{'2015': 853925,
 '2016': 810238,
 '2017': 192551,
 '2018': 799346,
 '2019': 820083,
 '2020': 394682,
 '2021': 364325,
 '2022': 564704,
 '2023': 333146}

In [14]:
league_id = 333146
game_key = 423
year = 2023

In [15]:
#make initial query scoped to individual fantasy football season
yahoo_query = YahooFantasySportsQuery(
    auth_dir,
    league_id= league_id,
    game_code='nfl',
    game_id= game_key
)

#from the query, get teams, managers, and manger ids while cleaning residual string gunk from query
teams = yahoo_query.get_league_teams()
team_ids_list = []
for team in teams:
    team_name = str(team.name)[1:].strip("'").strip('"')
    manager = team.managers[0].nickname
    id = team.team_id
    info = {'team_name': team_name, 'manager_nickname': manager, 'id': id}
    team_ids_list.append(info)

#stash team data in dataframe
team_ids_df = pd.DataFrame(team_ids_list)

In [16]:
# establish the maximum amount of weeks the season ran (via yahoo query method)
max_weeks = len(yahoo_query.get_game_info_by_game_id(game_key).game_weeks)

# iterate through each week, then each manager by id, and generate roster points for each
season_stats = []
print("Progress through week: ")
for week in range(1, max_weeks+1):
    print(f"|{week}|", end=" ")
    for manager_id in team_ids_df['id']:
        # raw player data by manager, week comes from query method
        try: 
            week_subquery = yahoo_query.get_team_roster_player_stats_by_week(manager_id, week)
        except:
            print(f"Yahoo Error occurred when making call for manager ID {manager_id}, week {week}")
            break
        else:
            # and is passed to custom method that organizes it into data we care about
            weekly_stats = generate_roster_points_by_week(week_subquery)
    
            # before adding the row, add the team, manager, and week information
            team_name = team_ids_df.loc[team_ids_df['id'] == manager_id, 'team_name'].item()
            weekly_stats['team_name'] = team_name
            weekly_stats['manager'] = map_teams_to_name(team_name)
            weekly_stats['week'] = week
            season_stats.append(weekly_stats)

#stash in a dataframe
season_stats_df = pd.DataFrame(season_stats)

Progress through week: 
|1| |2| |3| |4| |5| |6| |7| |8| |9| |10| |11| |12| |13| |14| |15| |16| |17| |18| 

2024-01-08 19:19:49.962 - ERROR - query.py - yfpy.query:205 - Attempt to retrieve data at URL https://fantasysports.yahooapis.com/fantasy/v2/team/423.l.333146.t.1/roster;week=18/players/stats?format=json failed with error: "Invalid week 18 specified in the URI."


Yahoo Error occurred when making call for manager ID 1, week 18


In [17]:
season_stats_df.to_csv(f'{year}_weekly_roster_stats.csv', index=False)